# Tensorflow and TensorRT to Frozen model

## Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.platform import gfile
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
%matplotlib inline
from PIL import Image as PILImage

# For import utils.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

from utils import label_map_util
from utils import visualization_utils as vis_util

import pprint
import time
import cv2

### For Jetson TX2

In [3]:
from keras import backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

Using TensorFlow backend.


## Variables of models
All models are in ./models folder

In [4]:
sub_dir = time.strftime("%Y_%m_%d-%H_%M_%S", time.localtime())
print(sub_dir)

2019_07_24-16_54_24


### 1.1) ssd_mobilenet_v1_coco_2017_11_17

In [ ]:
# What model to download.
MODEL_NAME = './models/ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
OUTPUT_NODES = ['num_detections', 'detection_boxes', 'detection_scores','detection_classes']

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
print(PATH_TO_LABELS)

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
print(PATH_TO_FROZEN_GRAPH)

### 1.2) faster_rcnn_resnet101_kitti_2018_01_28

In [15]:
# What model to download.
MODEL_NAME = './models/faster_rcnn_resnet101_kitti_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
OUTPUT_NODES = ['num_detections', 'detection_boxes', 'detection_scores','detection_classes']

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'kitti_label_map.pbtxt')
print(PATH_TO_LABELS)

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
print(PATH_TO_FROZEN_GRAPH)

data/kitti_label_map.pbtxt
./models/faster_rcnn_resnet101_kitti_2018_01_28/frozen_inference_graph.pb


### 1.3) aadc2018_frcnn_res101_200k_kitti

In [6]:
# What model to download.
MODEL_NAME = './models/aadc2018_frcnn_res101_200k_kitti'
OUTPUT_NODES = ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = MODEL_NAME + "/aadc_labels_2018_without_middlelane.pbtxt"
print(PATH_TO_LABELS)

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/aadc2018_frcnn_res101_200k_kitti.pb'
print(PATH_TO_FROZEN_GRAPH)

./models/aadc2018_frcnn_res101_200k_kitti/aadc_labels_2018_without_middlelane.pbtxt
./models/aadc2018_frcnn_res101_200k_kitti/aadc2018_frcnn_res101_200k_kitti.pb


## Download Model
from Internet

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper Function

#### 1) Function to read ".pb" model (TensorRT model is stored in ".pb")

In [9]:
# (can be used to read frozen model or TensorRT model)
def read_pb_graph(model):
    with gfile.FastGFile(model,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

#### 2) Function to load image into numpy array

In [10]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

#### 3) Function of showing graph

In [11]:
from IPython.display import clear_output, Image, display, HTML

# Helper functions for TF Graph visualization
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
    return strip_def
  
def show_graph(graph_def=None, max_const_size=32):
    """Visualize TensorFlow graph."""
    
    # If no input graph is given, get the default graph
    if graph_def is None:
        graph_def = tf.get_default_graph().as_graph_def()
        
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

#### 4) Showing image

In [12]:
def imshow(img):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg',img)
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

## Images variables
There are 2 images sources. Select one of them

### 1) demo images

In [ ]:
PATH_TO_TEST_IMAGES_DIR = './test_images2'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
pprint.pprint(TEST_IMAGE_PATHS)

### 2) Images for aadc2018_frcnn_res101_200k_kitti

In [13]:
PATH_TO_TEST_IMAGES_DIR = './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, file) for file in os.listdir(PATH_TO_TEST_IMAGES_DIR) if file.endswith(('jpg', 'png'))  ]
pprint.pprint(TEST_IMAGE_PATHS)

FINAL_CONFIG_FILE = PATH_TO_TEST_IMAGES_DIR + "/final_config.json"
print(FINAL_CONFIG_FILE)


['./models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1505828509427.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329368819.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329390645.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329398657.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329206644.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329401231.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329409963.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329204907.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329205476.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329401400.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329371996.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/

 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329394918.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329392695.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329373290.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329407350.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329402198.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329392989.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329401570.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329370204.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329367097.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1508416033791.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1534329365046.jpg',
 './models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/

## Tensorflow Inference (Detection)

### Load a (frozen) Tensorflow model into memory.
.pb file is in the folder named MODEL_NAME

In [16]:
tf.reset_default_graph()

In [17]:
graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:
        print('Read frozen model')
        tf_graph = read_pb_graph(PATH_TO_FROZEN_GRAPH)
#         show_graph(graph_def=tf_graph)
        tf.import_graph_def(tf_graph, name='')
    
        # write to tensorboard (check tensorboard for each op names)
        writer = tf.summary.FileWriter('./logs/'+sub_dir)
        writer.add_graph(sess.graph)
        writer.flush()
        writer.close()
        print("\nWrite logs {} success\n".format(sub_dir))
        
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            sess, # session
            tf.get_default_graph().as_graph_def(),# graph+weight from the session
            output_node_names=OUTPUT_NODES)

Read frozen model

Write logs 2019_07_24-16_54_24 success

INFO:tensorflow:Froze 0 variables.
INFO:tensorflow:Converted 0 variables to const ops.


### Function to inference single image

In [18]:
OUTPUTS = ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']

def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in OUTPUTS:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
#             print('tensor dict')
#             pprint.pprint(tensor_dict)
            
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[1], image.shape[2])
                detection_masks_reframed = tf.cast(
                tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            print("Run Inference")
            print("image size:",image.shape)
            output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image}) ## Boom here
            print("Finish Inference")
            
            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.int64)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
        return output_dict

### Inference

In [ ]:
total_time = 0

for idx, image_path in enumerate(TEST_IMAGE_PATHS):
    print(image_path)
    image = PILImage.open(image_path)
    image_np = load_image_into_numpy_array(image)
#     print(image_np.shape)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    
    # Actual detection.
    t1 = time.time()
    print("--")
    output_dict = run_inference_for_single_image(image_np_expanded, graph)
    print("---")
    t2 = time.time()
    delta_time = t2 - t1
    total_time += delta_time
    
#     print(output_dict)
    
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    
    print("----")
    # Saving image
    img_name = image_path.replace('test_images_20181027','detected_images')
    # for demo images
    if img_name == image_path:
        img_name = "./test_images2/%d.jpg"%idx
    print(idx,'- Saving images:', img_name, ",size:",image_np.shape, "used time:", delta_time)
    im = PILImage.fromarray(image_np, mode="RGB")
#     im.save("detected_images/"+img_name)
    im.save(img_name)
#     cv2.imwrite("./detected_images/"+idx+".jpg",image_np)
#     imshow(image_np)
#     break
    
print("Total time:",total_time)

./models/aadc2018_frcnn_res101_200k_kitti/test_images_20181027/1505828509427.jpg
--
Run Inference
image size: (1, 330, 1240, 3)


### Show deteced images

In [ ]:
PATH_TO_DETECTED_IMAGES_DIR = './test_images2'
DETECTED_IMAGE_PATHS = [ os.path.join(PATH_TO_DETECTED_IMAGES_DIR, file) for file in os.listdir(PATH_TO_DETECTED_IMAGES_DIR) if file.endswith("jpg") ]
print(DETECTED_IMAGE_PATHS)

In [ ]:
IMAGE_SIZE = (12, 8)
for idx, image_path in enumerate(DETECTED_IMAGE_PATHS):
    image = PILImage.open(image_path)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(np.asarray(image))

In [ ]:
# Or...
IMAGE_SIZE = (12, 8)
plt.figure(figsize=IMAGE_SIZE)
COLS = 1
ROWS = len(DETECTED_IMAGE_PATHS) / COLS+1

for idx, image_path in enumerate(DETECTED_IMAGE_PATHS):
    image = PILImage.open(image_path)
#     image.show()
    plt.subplot(ROWS, COLS, idx+1 )
    plt.imshow(np.asarray(image))

## Interferencing With TensorRT

### Optimize to TensorRT Model

In [ ]:
%%time
# outputs = [out.op.name for out in model.outputs] # name of output layer
BATCH_SIZE = 256

# convert (optimize) frozen model to TensorRT model
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,# frozen model
    outputs=OUTPUT_NODES,
    max_batch_size=BATCH_SIZE,# specify your max batch size
    max_workspace_size_bytes=2*(10**9),# specify the max workspace
    precision_mode="FP32") # precision, can be "FP32" (32 floating point precision) or "FP16"

In [ ]:
#write the TensorRT model to be used later for inference
pb_trt_filename = "trt_frozen_graph-"+sub_dir+".pb"
# PATH_TO_TRT_FROZEN_GRAPH = MODEL_NAME+ "/" + pb_trt_filename
PATH_TO_TRT_FROZEN_GRAPH = MODEL_NAME+"/trt_frozen_graph-2019_07_18-17_23_40.pb"
print(PATH_TO_TRT_FROZEN_GRAPH)

In [ ]:
with gfile.FastGFile(PATH_TO_TRT_FROZEN_GRAPH, 'wb') as f:
    f.write(trt_graph.SerializeToString())
print(pb_trt_filename + " is successfully stored!")

### Load TensorRT graph

In [ ]:
tf.reset_default_graph()

In [ ]:
trt_detection_graph = tf.Graph()
with trt_detection_graph.as_default():
    with tf.Session() as sess:
        print('Read TensorRT frozen model')
        trt_graph = read_pb_graph(PATH_TO_TRT_FROZEN_GRAPH)
#         show_graph(graph_def=tf_graph)
        tf.import_graph_def(trt_graph, name='')
#         print([node.name for node in trt_graph.node])
        
        # write to tensorboard (check tensorboard for each op names)
#         writer = tf.summary.FileWriter('./trt_logs/'+sub_dir)
#         writer.add_graph(sess.graph)
#         writer.flush()
#         writer.close()
#         print("\nWrite trt_logs success")

# print([node.name for node in trt_detection_graph.get_operations()])

### Inference

In [ ]:
trt_total_time = 0

for idx, image_path in enumerate(TEST_IMAGE_PATHS):
    image = PILImage.open(image_path)
    image_np = load_image_into_numpy_array(image)
#     print(image_np.shape)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    
    # Actual detection.
    t1 = time.time()
    print("--")
    output_dict = run_inference_for_single_image(image_np_expanded, trt_detection_graph)
    print("---")
    t2 = time.time()
    delta_time = t2 - t1
    trt_total_time += delta_time
    
#     print(output_dict)
    
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    # Showing image
    img_name = "%d.jpg"%idx
    print(idx,'- Saving images:', img_name, ",size:",image_np.shape, "used time:", delta_time)
    im = PILImage.fromarray(image_np, mode="RGB")
    im.save("detected_images/"+img_name)
#     cv2.imwrite("./detected_images/"+idx+".jpg",image_np)
#     imshow(image_np)
    if(idx==10):
        break
    
print("Total time:",trt_total_time)

## Different Time

In [ ]:
print("Detection time: ", total_time)
print("With TensorRT Detection time: ", trt_total_time)
print("Different time (TensorRT faster):", total_time - trt_total_time)
print("TensorRT improvement:", total_time/trt_total_time, "times")